In [73]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [74]:
url = "https://garuda.kemdikbud.go.id/documents?page={}&q=machine+learning"
mainurl = "https://garuda.kemdikbud.go.id"

# data = {
#     'name': [],
#     'published_at': [],
#     'abstract': [],
#     'publisher': [],
#     'web_publisher': [],
#     'company_publisher': [],
#     'link_publisher': [],
#     'category': [],
#     'link': []
# }

df = pd.DataFrame.from_dict({
    'name': [],
    'published_at': [],
    'abstract': [],
    'publisher': [],
    'web_publisher': [],
    'company_publisher': [],
    'link_publisher': [],
    'category': [],
    # 'link': [],
    # 'ris_citation': [], 
    # 'bibtex_citation': [], 
    # 'ori_source': [], 
    # 'ori_download': [], 
    # 'scholar': [],
})

for i in range(150):
    uri = f"{url.format(i)}"

    result = requests.get(url=uri).content
    doc = BeautifulSoup(result, "html.parser")
    article = doc.findAll('div', class_="article-item")

    for articles in article:
        articletitle = articles.find('a')
        href = articletitle.get('href')
        link_doc = mainurl + href
        spec = requests.get(url=link_doc).content

        journal = BeautifulSoup(spec, "html.parser")
        journal_name = journal.find('h3', class_="ui header").text
        journal_publish = journal.find('div', class_="four wide column").text
        journal_abstract = journal.find('div', class_="abstract-article").text
        
        segment = journal.find_all("div", class_="ui segment")
        publisher = journal.find("div", class_="ui segment j-art-display")
        name_publisher_meta = publisher.find("div", class_="j-title")
        web_publisher_meta = publisher.find("div", class_="j-website")
        pub_publisher_meta = publisher.find("div", class_="j-pub-name")
        category_publisher_meta = publisher.find("a", class_="ui tag label grey mini")
        name_publisher = name_publisher_meta.find("a").text
        web_publisher = web_publisher_meta.find("a")
        pub_publisher = pub_publisher_meta.find("a").text
        link_pub_publisher = pub_publisher_meta.find("a")

        web_publisher_fix = web_publisher['href']

        for div in segment:
            item_elements = div.find_all("div", class_="item")
            linked = []
            keys = ['ris_citation', 'bibtex_citation', 'ori_source', 'ori_download', 'scholar']
            for a in item_elements:
                anchor = a.find_all("a", class_="content")
                for index, link in enumerate(anchor):
                    href = link['href']
                    # print(href)
                    linked.append(href)
        
        new_list = [{key:value for key in keys} for value in linked]
        new_row = {'name': journal_name.replace("\n", ""),
            'published_at': journal_publish.replace("\n", ""),
            'abstract': journal_abstract.replace("\n", ""),
            'publisher': name_publisher.replace("\n", ""),
            'web_publisher': web_publisher_fix.replace("\n", ""),
            'company_publisher': pub_publisher.replace("\n", ""),
            'link_publisher': mainurl + link_pub_publisher['href'],
            'category': category_publisher_meta,
            # 'ris_citation': new_list[0]['ris_citation'],
            # 'bibtex_citation': new_list[0]['bibtex_citation'],
            # 'ori_source': new_list[0]['ori_source'],
            # 'ori_download': new_list[3]['ori_download'],
            # 'scholar': new_list[4]['scholar']
        }
        row_new_df = pd.DataFrame([new_row])
        df = pd.concat([df, row_new_df], ignore_index=True)
        # print(linked)

In [75]:
df.head(100)

,name,published_at,abstract,publisher,web_publisher,company_publisher,link_publisher,category
0,UNJUK KERJA METODE KLASIFIKASI SUPPORT VECTOR ...,Article InfoPublish Date 10 Mar 2015,Face recognition techniques and classification...,Jurnal Ilmiah Sinus,https://p3m.sinus.ac.id/jurnal/index.php/e-jur...,STMIK Sinar Nusantara Surakarta,https://garuda.kemdikbud.go.id/publisher/view/633,[Computer Science & IT]
1,Software Defect Prediction Using Synthetic Min...,Article InfoPublish Date 20 Feb 2020,Software testing is one of the crucial process...,Journal of Telematics and Informatics,http://section.iaesonline.com/index.php/JTI,IAES Indonesia Section and Partners,https://garuda.kemdikbud.go.id/publisher/view/65,[Computer Science & IT]
2,Peramalan Beban Listrik Jangka Pendek Mengguna...,Article InfoPublish Date 13 Sep 2012,Peramalan beban listrik jangka pendek merupaka...,Jurnal Teknik ITS,http://ejurnal.its.ac.id/index.php/teknik,Institut Teknologi Sepuluh Nopember Surabaya,https://garuda.kemdikbud.go.id/publisher/view/102,[Engineering]
3,Rain Prediction Using Rule-Based Machine Learn...,Article InfoPublish Date 01 May 2020,Rain prediction is an important topic that con...,"Advance Sustainable Science, Engineering and T...",http://journal.upgris.ac.id/index.php/asset,Universitas PGRI Semarang,https://garuda.kemdikbud.go.id/publisher/view/421,"[Chemical Engineering, Chemistry & Bioengineer..."
4,Prediction of 4f7-4f65d1 transition energy of ...,Article InfoPublish Date 09 Apr 2020,In order to establish a method to predict the ...,"Advance Sustainable Science, Engineering and T...",http://journal.upgris.ac.id/index.php/asset,Universitas PGRI Semarang,https://garuda.kemdikbud.go.id/publisher/view/421,"[Chemical Engineering, Chemistry & Bioengineer..."
...,...,...,...,...,...,...,...,...
95,Peramalan Pemakaian Air Pada PLTGU Di Pembangk...,Article InfoPublish Date 02 Jul 2018,Water is an absolute necessity every day that ...,Jurnal Pengembangan Teknologi Informasi dan Il...,http://j-ptiik.ub.ac.id/index.php/j-ptiik,Universitas Brawijaya,https://garuda.kemdikbud.go.id/publisher/view/28,[Computer Science & IT]
96,Penerapan Metode Extreme Learning Machine (ELM...,Article InfoPublish Date 15 May 2018,KHI Pipe Industries is a company that speciali...,Jurnal Pengembangan Teknologi Informasi dan Il...,http://j-ptiik.ub.ac.id/index.php/j-ptiik,Universitas Brawijaya,https://garuda.kemdikbud.go.id/publisher/view/28,[Computer Science & IT]
97,Optimasi Kandungan Gizi Susu Kambing Peranakan...,Article InfoPublish Date 14 Feb 2018,Milk is source of protein which is contain all...,Jurnal Pengembangan Teknologi Informasi dan Il...,http://j-ptiik.ub.ac.id/index.php/j-ptiik,Universitas Brawijaya,https://garuda.kemdikbud.go.id/publisher/view/28,[Computer Science & IT]
98,SUPERVISED MACHINE LEARNING MODEL FOR MICRORNA...,Article InfoPublish Date 30 Jun 2017,The cancer cell gene expression data in genera...,Jurnal Ilmu Komputer dan Informasi,http://jiki.cs.ui.ac.id/index.php/jiki,Universitas Indonesia,https://garuda.kemdikbud.go.id/publisher/view/36,[Computer Science & IT]


In [ ]:
df.to_csv('garuda_ml.csv', index=False)

In [ ]:
df.to_excel('garuda_ml.xlsx', index=False)